In [12]:
#导入必要的工具包
from selenium import webdriver
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from random import choice
import re
import pandas as pd

In [13]:
browser = webdriver.Chrome() # 打开谷歌浏览器
wait = ui.WebDriverWait(browser,10) # 设定最长等待加载时间为10秒
browser.get("http://s.weibo.com/") #打开微博搜索的地址

In [14]:
#搜索关键词的函数
def search(searchWord):
    wait.until(lambda browser: browser.find_element_by_class_name("gn_name")) #等待用户名出现，确认登录成功
    inputBtn = browser.find_element_by_tag_name("input") #找到搜索框
    inputBtn.click()
    inputBtn.send_keys(searchWord) #输入搜索关键词
    browser.implicitly_wait(10)
    browser.find_element_by_xpath("//button[@node-type='submit']").click() #点击“搜索”

In [15]:
#模拟登陆函数
def login(username,password):
    wait.until(lambda browser: browser.find_element_by_xpath("//a[@node-type='loginBtn']"))#找到左上角登录按钮
    browser.find_element_by_xpath("//a[@node-type='loginBtn']").click() #点击登录按钮
    wait.until(lambda browser: browser.find_element_by_xpath("//input[@name='username']")) #等待登录页面出现
    user = browser.find_element_by_xpath("//input[@name='username']")
    user.clear() 
    user.send_keys(username) #输入用户名
    psw = browser.find_element_by_xpath("//input[@name='password']")
    psw.clear()
    psw.send_keys(password) #输入密码
    browser.find_element_by_xpath("//a[@node-type='submitBtn']").click() #点击“登录”

In [16]:
#模拟登录操作
login('xxxxxx','xxxxxx')

In [17]:
#爬虫获取信息的函数
def gettext(stressword,skinword):
    
    names=[]
    stresswords=[]
    skinwords=[]
    dates=[]
    sources=[]
    contents =[]
    forwards=[]
    comments=[]
    likes=[]
    

    #展开微博全文
    full_content=browser.find_elements_by_link_text("展开全文")
    for i in full_content:
        i.click()
    
    #获取用户姓名
    name=browser.find_elements_by_xpath("//div[@class='card-feed']/div[@class='content']/div[@class='info']")
    
    for i in name:
        stresswords.append(stressword)
        skinwords.append(skinword)
        names.append(i.text)
    
    #获取日期和超链接
    date_source=browser.find_elements_by_xpath("//div[@class='card-feed']//div[@class='content']/p[@class='from']/a[@target='_blank']")
    
    for i in date_source:
        dates.append(i.text)
        sources.append(i.get_attribute('href'))
    
    #获取文本内容
    content = browser.find_elements_by_xpath("//p[@node-type='feed_list_content']")
    
    for n in content:
        try:
            emojiToDie = re.compile(u'[\U00010000-\U0010ffff]')
        except re.error:
            emojiToDie = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')
        mytext = emojiToDie.sub(u'', n.text) ##emojiToDie 顾名思义，是为了解决微博文本内容里的emoji图像而建立的一个正则表达式，我为什么这么恨emoji呢，因为你在往数据库里存，在解析它的时候，都会报错，所以干脆给它过滤掉
        contents.append(mytext) ##把每一条微博加到容器里
    
    #获取转赞评
    forward=browser.find_elements_by_xpath("//div[@class='card-act']/ul/li/a[@action-type='feed_list_forward']")
    comment=browser.find_elements_by_xpath("//div[@class='card-act']/ul/li/a[@action-type='feed_list_comment']")
    like=browser.find_elements_by_xpath("//div[@class='card-act']/ul/li/a[@action-type='feed_list_like']")
    
    for i in range(0, len(forward)):
        forwards.append(forward[i].text)
        comments.append(comment[i].text)
        likes.append(like[i].text)
        
    
    all_text=list(zip(stresswords,skinwords,names,dates,contents,forwards,comments,likes,sources))
    
    return all_text

In [18]:
#翻页函数
def nextPage(stressword, skinword):
    full=[]
    browser.get('https://s.weibo.com/weibo?q={}%20{}&scope=ori&suball=1&timescope=custom:2020-03-10-0:2020-04-10-0&Refer=SWeibo_box&page={}'.format(stressword, skinword,1))
    length=len(browser.find_elements_by_xpath("//div[@class='m-page']/div/span/ul/li"))
    for i in range(1,length+1):
        browser.implicitly_wait(10)
        
        browser.get('https://s.weibo.com/weibo?q={}%20{}&scope=ori&suball=1&timescope=custom:2020-03-10-0:2020-04-10-0&Refer=SWeibo_box&page={}'.format(stressword, skinword,i))
        content=gettext(stressword, skinword)
        full+=content
    result=pd.DataFrame(full,columns=['压力词','皮肤词','用户名','日期','内容','转发数','评论数','点赞数','数据源'])
    return result

In [21]:
#搜索关键词存入列表
searchword_stress=['宝宝','孩子','孕期','父母','爸妈','催婚']
searchword_skin=['皮肤','痘','过敏','干燥','皱纹','闭口','黑眼圈','衰老','毛孔','湿疹','松弛','粉刺','皮炎','黑头','细纹']

In [20]:
#根据关键词运行爬虫函数并储存
for i in searchword_stress:
    for j in searchword_skin:
        a=nextPage(i,j)
        a.to_excel('{}_{}.xlsx'.format(i,j))